In [5]:
import cv2
import os
import tensorflow as tf
import numpy as np
import opendatasets as od

import keras


In [6]:
print(os.getcwd())
if 'Emotions' not in os.getcwd():
    os.chdir(os.getcwd() + '/Emotions-ImageClassification-TensorFlow')

import src

/teamspace/studios/this_studio


In [7]:
os.getcwd()

'/teamspace/studios/this_studio/Emotions-ImageClassification-TensorFlow'

In [8]:
config = src.read_yaml('config/config.yaml')
print(config)
od.download(config['data_ingestion']['url_download'])

{'data_ingestion': {'url_download': 'https://www.kaggle.com/datasets/msambare/fer2013', 'data_path': 'dataset/'}, 'data_transformation': {'image_width': 32, 'image_height': 32, 'image_channels': 1, 'num_classes': 6, 'train_ratio': 0.8, 'val_ratio': 0.1, 'test_ratio': 0.1, 'save_path': 'data_compressed/'}, 'data_loader': {'load_path': 'data_compressed/', 'batch_size': 32}}
Skipping, found downloaded files in "./fer2013" (use force=True to force download)


In [9]:
data_dir = 'fer2013'

In [10]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True,
)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    directory=data_dir + '/train', 
    target_size=(32, 32),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True
)
val_generator = val_datagen.flow_from_directory(
    directory=data_dir + '/test',
    target_size=(32, 32),
    batch_size = 64,
    color_mode = "grayscale",
    class_mode = "categorical",
    shuffle=True
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from keras import metrics

In [12]:
model= tf.keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32, 32, 1)))
model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) 
model.add(Dense(256,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
    
model.add(Dense(512,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

model.compile(
    optimizer = Adam(learning_rate=0.0001), 
    loss='categorical_crossentropy', 
    metrics=[metrics.CategoricalAccuracy(),
            metrics.AUC(),
            metrics.Precision(),
            metrics.Recall(),
            metrics.F1Score(average="weighted"),]
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
history = model.fit(train_generator, epochs = 10, validation_data = val_generator)

Epoch 1/10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1715161208.897798   50659 service.cc:145] XLA service 0x7f0f10002940 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715161208.897838   50659 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-05-08 09:40:09.078015: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-08 09:40:09.564545: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


  5/449 ━━━━━━━━━━━━━━━━━━━━ 13s 30ms/step - auc_1: 0.5066 - categorical_accuracy: 0.1368 - f1_score: 0.1472 - loss: 9.9587 - precision_1: 0.1299 - recall_1: 0.0629 

I0000 00:00:1715161221.662849   50659 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


449/449 ━━━━━━━━━━━━━━━━━━━━ 48s 68ms/step - auc_1: 0.5427 - categorical_accuracy: 0.1752 - f1_score: 0.1839 - loss: 9.5448 - precision_1: 0.1856 - recall_1: 0.0699 - val_auc_1: 0.5137 - val_categorical_accuracy: 0.1899 - val_f1_score: 0.1244 - val_loss: 8.7692 - val_precision_1: 0.2407 - val_recall_1: 0.0493
Epoch 2/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - auc_1: 0.5922 - categorical_accuracy: 0.2073 - f1_score: 0.2066 - loss: 8.3231 - precision_1: 0.2293 - recall_1: 0.0538 - val_auc_1: 0.6916 - val_categorical_accuracy: 0.2881 - val_f1_score: 0.2115 - val_loss: 7.2026 - val_precision_1: 0.5297 - val_recall_1: 0.0609
Epoch 3/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - auc_1: 0.6366 - categorical_accuracy: 0.2465 - f1_score: 0.2390 - loss: 7.1666 - precision_1: 0.2989 - recall_1: 0.0643 - val_auc_1: 0.7036 - val_categorical_accuracy: 0.3072 - val_f1_score: 0.2579 - val_loss: 6.1557 - val_precision_1: 0.5415 - val_recall_1: 0.0591
Epoch 4/10
449/449 ━━━━━━━━━━━━━━━━━━━━ 16s 3

KeyboardInterrupt: 